<a href="https://colab.research.google.com/github/Saputoa21/Machine-Translation/blob/main/filterMT_AdvancedMT_2025W.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://github.com/ymoslem/Adaptive-MT-LLM-Fine-tuning/blob/main/Data-Processing-Adaptive-MT.ipynb
#https://github.com/ymoslem/MT-Preparation/tree/main

In [ ]:
!git clone https://github.com/ymoslem/MT-Preparation.git

fatal: destination path 'MT-Preparation' already exists and is not an empty directory.


In [ ]:
%cd MT-Preparation

/MT-Preparation


In [ ]:
!pip3 install --user -r requirements.txt

In [ ]:
%cd ..

/


In [ ]:
!ls

bin			    kaggle	    NGC-DL-CONTAINER-LICENSE  srv
boot			    lib		    opt			      sys
content			    lib32	    proc		      tmp
cuda-keyring_1.1-1_all.deb  lib64	    python-apt		      tools
datalab			    libx32	    python-apt.tar.xz	      usr
dev			    media	    root		      var
etc			    mnt		    run
home			    MT-Preparation  sbin


In [ ]:
!python3 MT-Preparation/filtering/filter.py train.en-de.en train.en-de.de en de

Traceback (most recent call last):
  File "//MT-Preparation/filtering/filter.py", line 152, in <module>
    prepare(source_file, target_file, source_lang, target_lang, lower=False)
  File "//MT-Preparation/filtering/filter.py", line 21, in prepare
    df_source = pd.read_csv(source_file, names=['Source'], sep="\0", quoting=csv.QUOTE_NONE, skip_blank_lines=False, on_bad_lines="skip")
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py",

In [ ]:
!python3 MT-Preparation/filtering/semantic_filter.py train.en-de.en-filtered.en train.en-de.de-filtered.de en de 10000 0.45

Traceback (most recent call last):
  File "//MT-Preparation/filtering/semantic_filter.py", line 3, in <module>
    from sentence_transformers import SentenceTransformer
  File "/usr/local/lib/python3.12/dist-packages/sentence_transformers/__init__.py", line 15, in <module>
    from sentence_transformers.cross_encoder import (
  File "/usr/local/lib/python3.12/dist-packages/sentence_transformers/cross_encoder/__init__.py", line 3, in <module>
    from .CrossEncoder import CrossEncoder
  File "/usr/local/lib/python3.12/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py", line 16, in <module>
object address  : 0x7cd1c6228d60
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [ ]:
!wc -l *

wc: model_cache: Is a directory
       0 model_cache
wc: MT-Preparation: Is a directory
       0 MT-Preparation
wc: sample_data: Is a directory
       0 sample_data
   50000 train.en-de.de
   48315 train.en-de.de-filtered.de
   47614 train.en-de.de-filtered.de.semantic.de
   50000 train.en-de.en
   48315 train.en-de.en-filtered.en
   47614 train.en-de.en-filtered.en.semantic.en
  291858 total


# **My corpus text**


I have chosen MultiUN (v1) corpus with the language pair EN-RU and size of ... sentences. I splitted it into train () and test () sets.

The files are:

*   UN-train.en-ru.en
*   UN-train.en-ru.ru
*   UN-test.en-ru.en
*   UN-test.en-ru.ru

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file1 = "/content/drive/MyDrive/file1.txt.gz"
file2 = "/content/drive/MyDrive/file2.txt.gz"

In [ ]:
import gzip

def extract_first_n_lines(gz_path, n_lines=100000):
    lines = []
    with gzip.open(gz_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= n_lines:
                break
            lines.append(line)
    print(f"Extracted {len(lines):,} lines from {gz_path}")
    return lines

# Example use:
lines1 = extract_first_n_lines(file1, n_lines=100000)
lines2 = extract_first_n_lines(file2, n_lines=100000)

In [ ]:
import random

def split_data(lines, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, shuffle=True, prefix="my_corpus"):
    if shuffle:
        print("Shuffling lines...")
        random.shuffle(lines)

    n = len(lines)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)

    train = lines[:n_train]
    val = lines[n_train:n_train + n_val]
    test = lines[n_train + n_val:]

    def save(name, data):
        with open(f"{prefix}_{name}.txt", "w", encoding="utf-8") as f:
            f.writelines(data)
        print(f"Saved {prefix}_{name}.txt ({len(data):,} lines)")

    save("train", train)
    save("val", val)
    save("test", test)

In [ ]:
split_data(lines1, prefix="sample_corpus")

In [ ]:
split_data(lines2, prefix="sample_corpus")

In [ ]:
lang_pair = "en-ru"

train_src = "UN-train.en-ru.en"
train_trg = "UN-train.en-ru.ru"
train_json_UN = "UN-v1.en-ru.train.json"

txt2json(lang_pair, train_src, train_trg, train_json_UN) #create a json for training

In [ ]:
with open("UN-v1.en-ru.train.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]
    print(f"number of samples: {len(data)}")
    print("first sample:", data[0])

In [ ]:
subset = data[:20000]

with open("UN-v1.en-ru.train.50k.json", "w", encoding="utf-8") as f:
    for item in subset:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print("Saved subset to UN-v1.en-ru.train.50k.json")

In [ ]:
with open("UN-v1.en-ru.train.50k.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]
    print(f"number of samples: {len(data)}")
    print("first sample:", data[0])

In [ ]:
!git clone https://github.com/ymoslem/MT-Preparation.git

Cloning into 'MT-Preparation'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 323 (delta 35), reused 21 (delta 20), pack-reused 268 (from 2)
Receiving objects: 100% (323/323), 94.95 KiB | 1.48 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [ ]:
%cd MT-Preparation

/content/MT-Preparation


In [ ]:
!pip3 install --user -r requirements.txt

In [ ]:
%cd ..

/content


In [ ]:
!ls

MT-Preparation		    train.en-de.de-filtered.de.semantic.de
sample_data		    train.en-de.en
train.en-de.de		    train.en-de.en-filtered.en
train.en-de.de-filtered.de  train.en-de.en-filtered.en.semantic.en


In [ ]:
!python3 MT-Preparation/filtering/filter.py train.en-de.en train.en-de.de en de

Dataframe shape (rows, columns): (50000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 49997
--- Duplicates Deleted			--> Rows: 49997
--- Source-Copied Rows Deleted		--> Rows: 49997
--- Too Long Source/Target Deleted	--> Rows: 48399
--- Too Short Source/Target Deleted	--> Rows: 48321
--- HTML Removed			--> Rows: 48321
--- Rows will remain true-cased		--> Rows: 48321
--- Rows with Empty Cells Deleted	--> Rows: 48315
--- Rows Shuffled			--> Rows: 48315
--- Source Saved: train.en-de.en-filtered.en
--- Target Saved: train.en-de.de-filtered.de
